# 📦 Importing Required Libraries

# os       → For interacting with the file system (e.g., listing files, paths)
# pandas   → For reading and manipulating CSV data in tabular form
# numpy    → For numerical operations and efficient array handling
# sklearn.model_selection.train_test_split → For splitting the dataset into training and testing sets
# sklearn.ensemble.RandomForestClassifier  → The machine learning model used for drought classification
# sklearn.metrics → To evaluate the model using confusion matrix and classification report (precision, recall, F1-score)
# joblib   → For saving and loading the trained model efficiently


In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# 📂 Loading and Combining Raw Monthly Reservoir CSV Files

# Step 1: Define the folder path where all 24 monthly CSV files are stored
folder_path = "/content/datasets"

# Step 2: List all CSV files in that folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Step 3: Loop through each CSV file, read it into a DataFrame, and store it in a list
df_list = []
for file in csv_files:
    full_path = os.path.join(folder_path, file)   # Create full path to the file
    df = pd.read_csv(full_path)                   # Read the CSV file
    df['source_file'] = file                      # (Optional) Track source file for reference/debugging
    df_list.append(df)                            # Add DataFrame to the list

In [ ]:
folder_path = "/content/datasets"
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

df_list = []
for file in csv_files:
    full_path = os.path.join(folder_path, file)
    df = pd.read_csv(full_path)
    df['source_file'] = file  # Optional
    df_list.append(df)

# 🔗 Merging All Monthly DataFrames into One

# - `pd.concat(...)` combines all DataFrames in `df_list` into a single large DataFrame (`combined_df`).
# - `ignore_index=True` resets the row index, so the merged DataFrame has a continuous index.
# - `df = combined_df.copy()` creates a working copy of the combined data for further processing.
# - `print(len(df))` and `len(combined_df)` show the total number of records loaded from all files.

# This gives us one unified dataset that will be used for feature engineering and modeling.

In [ ]:
combined_df = pd.concat(df_list, ignore_index=True)
df = combined_df.copy()
print(len(df))
len(combined_df)

7093


<ipython-input-7-d23a151b379b>:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(df_list, ignore_index=True)


7093

# 🧹 Data Cleaning: Column Name Standardization

# - Strips leading/trailing whitespaces from column names
# - Converts all column names to lowercase
# - Replaces spaces with underscores for consistency

# ❌ Remove Incomplete or Invalid Records

# - Drops rows where any of the essential columns are missing (NaN)
# - Filters out rows with 0 or negative capacity, or invalid gross storage values


In [ ]:
# 2. Clean Data
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Drop rows with critical missing values
df = df.dropna(subset=['capacity', 'gross_storage', 'frl', 'level_in_feet_present', 'inflow', 'outflow'])

# Remove invalid or zero values
df = df[(df['capacity'] > 0) & (df['gross_storage'] >= 0)]

# 🛠️ Feature Engineering

# - storage_percent: How full the reservoir is
# - net_flow: Difference between inflow and outflow
# - flow_ratio: Ratio of inflow to (outflow + 1) to avoid divide-by-zero
# - net_flow_per_capacity: Normalized net flow relative to capacity



In [ ]:
# Calculate useful features
df['storage_percent'] = (df['gross_storage'] / df['capacity']) * 100
df['net_flow'] = df['inflow'] - df['outflow']
df['flow_ratio'] = df['inflow'] / (df['outflow'] + 1)  # +1 to avoid division by zero
df['net_flow_per_capacity'] = df['net_flow'] / df['capacity']

# UPDATED: Define drought based on new logic
df['is_drought'] = df.apply(lambda row: 1 if (row['storage_percent'] < 30 or row['net_flow'] < 0) else 0, axis=1)

# 🚩 Drought Label Definition

# - A reservoir is labeled as drought (1) if:
#   → storage_percent < 30% (moderate drought)
#   OR
#   → net_flow is negative (more outflow than inflow)
# - Otherwise, it's labeled as non-drought (0)

# 🎯 Feature Selection and Target Variable Setup

# - Selects relevant columns as features for the model
# - Sets `is_drought` as the target label


In [ ]:
features = [
    'capacity',
    'level_in_feet_present',
    'gross_storage',
    'inflow',
    'outflow',
    'net_flow',
    'flow_ratio',
    'net_flow_per_capacity'
]

X = df[features]
y = df['is_drought']

# ✂️ Splitting Data for Training and Testing

# - 80% for training, 20% for testing
# - `stratify=y` ensures class distribution (drought vs non-drought) is preserved
# - `random_state=42` ensures reproducibility


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 🌲 Train Random Forest Classifier

# - Uses 100 decision trees
# - Random state set for reproducibility


In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

# 📊 Model Predictions and Performance Metrics

# - Predict drought status for test set
# - Print classification metrics: precision, recall, F1-score
# - Display confusion matrix to visualize correct and incorrect classifications


In [ ]:
y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       356
           1       1.00      1.00      1.00      1054

    accuracy                           1.00      1410
   macro avg       1.00      1.00      1.00      1410
weighted avg       1.00      1.00      1.00      1410

Confusion Matrix:
[[ 356    0]
 [   1 1053]]


# 💾 Save Trained Model and Cleaned Data

# - Saves the trained model as a .pkl file using joblib
# - Saves the cleaned and labeled dataset to a CSV for future use

In [ ]:
joblib.dump(model, "drought_prediction_model.pkl")
df.to_csv("cleaned_reservoir_dataset.csv", index=False)

print("✅ Model training complete and saved as 'drought_prediction_model'")

✅ Model training complete and saved as 'drought_prediction_model'
